# News Articles Summarization with Distilled Bert

### Setting up the environment

In [33]:
! pip install evaluate

In [34]:
! pip install rouge_score

### Importing Libraries

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
import numpy as np
from datasets import load_dataset
import evaluate

### Loading Dataset

In [21]:
data = pd.read_csv('E:\GBC\Deep Learning 1\Project\data\merge_df.csv')

In [22]:
data.head()

,Unnamed: 0,File_path,Articles,Summaries
0,0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."


In [23]:
data.rename(columns={'File_path':'Category'}, inplace=True)

In [24]:
data.head()

,Unnamed: 0,Category,Articles,Summaries
0,0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."


In [25]:
data.shape

(5449, 4)

In [26]:
data['Category'].value_counts()

Category
business         1228
politics         1158
sport            1021
entertainment     925
tech              802
crime             110
lifestyle          78
law                41
sports             30
science            25
technology         18
architecture        4
accidents           4
art                 2
health              2
environment         1
Name: count, dtype: int64

In [27]:
data = data[data['Category'] == 'entertainment'].reset_index(drop=True)

In [28]:
data.drop(columns=['Unnamed: 0','Category'], inplace=True)

In [29]:
data.head()

,Articles,Summaries
0,Super Size Me wins writers' award..Super Size ...,Spurlock was given his award on the same day t...
1,"Mogul Wilson backing UK rap band..Tony Wilson,...","Tony Wilson, the music mogul who established t..."
2,Police praise 'courageous' Ozzy..Rock star Ozz...,"""I could have been badly injured or shot or an..."
3,Eastwood's Baby scoops top Oscars..Clint Eastw...,The boxing drama was named best picture and Ea...
4,Actor Scott is new Bond favourite..Bookmaker W...,Bookmaker William Hill has stopped taking bets...


In [30]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [46]:
train_ds = Dataset.from_pandas(data_train, preserve_index=False)
test_ds = Dataset.from_pandas(data_test, preserve_index=False)

In [47]:
articles = DatasetDict()

In [48]:
articles['train'] = train_ds
articles['test'] = test_ds

In [49]:
articles

DatasetDict({
    train: Dataset({
        features: ['Articles', 'Summaries'],
        num_rows: 832
    })
    test: Dataset({
        features: ['Articles', 'Summaries'],
        num_rows: 93
    })
})

### Data Pre-processing

In [50]:
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 10
OUT_DIR = 'results_t5base'
MAX_LENGTH = 512 

In [51]:
dataset_train = articles['train']
dataset_valid = articles['test']

In [52]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [53]:
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['Articles']]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )
 
    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summaries']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )
 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [54]:
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [55]:
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [56]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [57]:
rouge = evaluate.load("rouge")

In [58]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [59]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [60]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=200,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0003,
    dataloader_num_workers=4
)

In [61]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

In [62]:
history = trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
200,0.478800,0.234253,0.915700,0.852700,0.901800,191.010800
400,0.309000,0.200994,0.925700,0.866800,0.913300,191.204300
600,0.391700,0.185056,0.931900,0.878400,0.920400,191.204300
800,0.284900,0.175317,0.935700,0.885200,0.925000,191.204300
1000,0.273700,0.173083,0.937600,0.889600,0.927700,191.204300


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [64]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.17307457327842712,
 'eval_rouge1': 0.9376,
 'eval_rouge2': 0.8891,
 'eval_rougeL': 0.9275,
 'eval_gen_len': 191.2043,
 'eval_runtime': 23.5104,
 'eval_samples_per_second': 3.956,
 'eval_steps_per_second': 0.51,
 'epoch': 10.0}

In [65]:
trainer.save_model("/kaggle/working/t5-base-entertainment-summarizer")

### Inference

In [39]:
article = data_test.iloc[0,0]
article

'Belle named \'best Scottish band\'..Belle & Sebastian have been named the best Scottish band of all time after a three month-long public poll...The group beat Travis and Idlewild into second and third place respectively. Franz Ferdinand, who recently picked up five Brit Award nominations, ended up in 15th place, while the Eurythmics wound up at a lowly 38. Other Scottish acts, such as the Mull Historical Society who also featured in the top 50, performed at a party in Glasgow where the result was announced. Scottish-based band Snow Patrol, who finished 14th in the vote and have been nominated for a pair of Brit Awards, were among the performers who covered well-known Scottish pop songs at the party on Wednesday night...Indie stalwarts Belle & Sebastian have enjoyed a chart career stretching back to 1997. They were the surprise winners of the Brit Award for best breakthrough act two years later. Scottish bands from earlier musical eras also made it into the final list, including 1970s 

In [40]:
summary = data_test.iloc[0,1]

In [47]:
summary

'Belle & Sebastian have been named the best Scottish band of all time after a three month-long public poll.Scottish magazine The List recently compiled a list of the top 50 Scottish bands of all time, but left the final decision to the public.Scottish bands from earlier musical eras also made it into the final list, including 1970s tartan boy band the Bay City Rollers and goth favourites the Jesus and Mary Chain.Scottish-based band Snow Patrol, who finished 14th in the vote and have been nominated for a pair of Brit Awards, were among the performers who covered well-known Scottish pop songs at the party on Wednesday night.BBC Radio Scotland presenter Vic Galloway, who has been involved in the project, said it had been "great fun" to look back at Scotland\'s musical heritage and take note of up-and-coming Scottish acts.'

In [3]:
model = T5ForConditionalGeneration.from_pretrained('TJ - t5-base-entertainment-summarizer', use_safetensors=True)

In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

c:\Users\eltig\anaconda3\envs\maths\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eltig\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


c:\Users\eltig\anaconda3\envs\maths\lib\site-packages\transformers\models\t5\tokenization_t5.py:217: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so no

In [41]:
inputs =  tokenizer(
        article,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

In [42]:
outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)

In [43]:
predicted_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted Summary:", predicted_summary)

Predicted Summary: 'best Scottish band' Belle & Sebastian have been named the best Scottish band of all time after a three month-long public poll.Scottish magazine The List recently compiled a list of the top 50 Scottish bands of all time, but left the final decision to the public.Scottish-based band Snow Patrol, who finished 14th in the vote and have been nominated for a pair of Brit Awards, were among the performers who covered well-known Scottish pop songs at the party on Wednesday night.Scottish band Snow Patrol, who finished 14th in the vote and have been nominated for a pair of Brit Awards, were among the performers who covered well-known Scottish pop songs at the party on Wednesday night.Scottish magazine The List recently compiled a list of the top 50 Scottish bands of all time, but left the final decision to the public.


In [44]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [45]:
outputs_base = base_model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)

In [46]:
predicted_summary_base = tokenizer.decode(outputs_base[0], skip_special_tokens=True)
print("Predicted Summary:", predicted_summary_base)

Predicted Summary: Belle & Sebastian have been named the best Scottish band of all time. the group beat Travis and Idlewild into second and third place respectively. other bands from earlier musical eras also made it into the final list.
